In [24]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")

Agents in essence are LLM's that accomplish a specific task. They can be supplemented with tools,structured output, and handoff to other agents.

In [ ]:
from agents import Agent, Runner, trace

agent = Agent(
    name = "Basic Agent",
    instructions = "You are a helpful assistant.Respond all in Caps",
    
    )

result = await Runner.run(agent, "who is narendra modi?")
result.final_output

In [12]:
cricket_agent = Agent(
    name = "Cricket Agent",
    instructions="You are cricket assistant. If the question is related to the cricket only respond otherwise respond in a generic manner to give the error.",
    model = "gpt-4o-mini"

)

result = await Runner.run(cricket_agent, "who is MS DHONI?")
result.final_output

'Mahendra Singh Dhoni, commonly known as MS Dhoni, is a former Indian cricketer and captain of the Indian national team. He is regarded as one of the greatest wicket-keeper batsmen and captains in the history of cricket. Dhoni led India to numerous significant victories, including the ICC T20 World Cup in 2007 and the ICC Cricket World Cup in 2011. He is known for his calm demeanor, strategic thinking, and exceptional finishing abilities in matches. After retiring from international cricket in 2020, he continues to play in the Indian Premier League (IPL) for the Chennai Super Kings (CSK).'

In [13]:
language_agent = Agent(
    name = "Language Agent",
    instructions="you are a language expert. You are given a information related to cricket and you need to rewrite it in a different language"
)

cricket_result = await Runner.run(cricket_agent, "who is MS DHONI?")
translated_result = await Runner.run(language_agent, f"Translate this cricket information into Telugu : {cricket_result.final_output}")
print(translated_result.final_output)

మహేంద్ర సింగ్ ధోనిని సాధారణంగా ఎం.ఎస్. ధోని అని పేరు పిలుస్తారు. అతను భారత క్రికెట్ జట్టు మాజీ క్రికెటర్ మరియు కెప్టెన్. ఆయన అసాధారణ నాయకత్వం, వికెట్-కీపింగ్ నైపుణ్యాలు, మరియు పరిమిత ఓవర్ల క్రికెట్‌లో ముగింపు సామర్థ్యాలకు ప్రసిద్ధి. 2007లో ICC T20 ప్రపంచ కప్, 2011లో ICC క్రికెట్ ప్రపంచ కప్, మరియు 2013లో ICC ఛాంపియన్స్ ట్రోఫీని గెలవడంలో దూకుడు చూపించాడు. భారత ప్రీమియర్ లీగ్ (IPL)లో కూడా అతని ప్రత్యేక స్థానం ఉంది. చెన్నై సూపర్ కింగ్స్ (CSK) కి అనేక ఛాంపియన్‌షిప్‌లు అందించాడు. ధోని క్రికెట్ చరిత్రలో ఒక గొప్ప వికెట్-కీపర్ బ్యాట్స్‌మాన్ మరియు కెప్టెన్‌గా పేరుపొందాడు.


Structured outputs are a way to format the output of an llm in a structured manner.

In [14]:
from pydantic import BaseModel
from agents import Agent

class PlayerHistory(BaseModel):
    player_name : str
    runs_in_odi : int
    avg_in_odi : float
    runs_in_T20 : int
    avg_in_T20 : float
    runs_in_test : int
    avg_in_test : float
    runs_in_ipl : int
    avg_in_ipl : float
    highest_score_in_odi : int
    highest_score_in_T20 : int
    highest_score_in_test : int
    highest_score_in_ipl : int


cric_agent = Agent(
    name = "cric agnet",
    instructions = "you are a expert cricket agent when ever i entered a player name. you should output that as an actual desired player history",
    output_type = PlayerHistory
)

result = await Runner.run(cric_agent, "MS dhoni")
result.final_output

PlayerHistory(player_name='MS Dhoni', runs_in_odi=10773, avg_in_odi=50.57, runs_in_T20=1617, avg_in_T20=37.6, runs_in_test=4876, avg_in_test=38.09, runs_in_ipl=4878, avg_in_ipl=39.79, highest_score_in_odi=183, highest_score_in_T20=56, highest_score_in_test=224, highest_score_in_ipl=84)

In [ ]:
from agents import agent, Runner, function_tool
from pydantic import BaseModel
from agents import WebSearchTool

class MovieCast(BaseModel):
    movie_name : str
    hero_name : str
    heroine_name : str
    budget : float
    released_date : str

@function_tool
def movie_release_date(movie: str) -> str:
    print(f"Getting Movie release date for {movie}")
    release_dates = {
         "hari hara veera mallu" : "24-07-2025"
    }
    return release_dates.get(movie.lower(), "TBA")

movie_agent = Agent(
    name = "movie agent",
    instructions= ( "You are a movie expert. When a user gives a movie name, "
        "If you not found the released date. You MUST use the `get_release_date` tool to get the release date. "
        "Respond in the MovieCast format."),
    tools=[movie_release_date, 
           WebSearchTool()],
    output_type = MovieCast,
    model = "gpt-4o"
)

while True:
    query = input("Enter the movie name")
    if query.lower() == "quit":
        break
    result = await Runner.run(movie_agent, query)
    print("QUERY", query)
    print("Answer", result.final_output)

Getting Movie release date for hari hara veera mallu
QUERY hari hara veera mallu
Answer movie_name='Hari Hara Veera Mallu' hero_name='Pawan Kalyan' heroine_name='Nidhhi Agerwal' budget=150.0 released_date='TBA'


Handoffs - A way for an agents to invoke another agent

In [16]:
from agents import Agent , Runner
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline : str
    tutorial : str

tutorial_generator = Agent(
    name = "tutorial-generator",
    handoff_description="Used for generating a tutorial based on an outline.",
    instructions=(
        "Given a programming topic and a outline, your job is to generate code snippets for each section of the outline.",
        "Format the tutorial in markdown using a mix of text for explanation and code snippets for examples",
        "Where it makes sense, include comments in the code snippets to further explain the code"
        ),
    output_type=Tutorial
)

outline_builder = Agent(
    name = "Outline-Builder",
    instructions=(
                "Given a particular programming topic, your job is to help come up with a tutorial. you will do that by crafting an outline.",
                "After making the outline, hand it to the tutorial generator agent"
                  ),
        
    
    handoffs=[tutorial_generator]
)


tutorial_response = await Runner.run(outline_builder, "inheritence in python")
print(tutorial_response.final_output)

Instructions must be a string or a function, got ('Given a particular programming topic, your job is to help come up with a tutorial. you will do that by crafting an outline.', 'After making the outline, hand it to the tutorial generator agent')


Inheritance in Python is a fundamental concept in object-oriented programming that allows a class (known as a child or derived class) to inherit attributes and methods from another class (known as a parent or base class). This promotes code reusability and establishes a relationship between different classes.

### Basic Concepts

1. **Parent Class**: The class whose properties and methods are inherited.
2. **Child Class**: The class that inherits from the parent class.

### Syntax

```python
class ParentClass:
    # Parent class methods and attributes

class ChildClass(ParentClass):
    # Child class methods and attributes
```

### Example

```python
class Animal:
    def __init__(self, name):
        self.name = name

    def speak(self):
        print(f"{self.name} makes a sound")

class Dog(Animal):
    def speak(self):
        print(f"{self.name} barks")

class Cat(Animal):
    def speak(self):
        print(f"{self.name} meows")

# Usage
dog = Dog("Buddy")
dog.speak()  # Output: B

In [17]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

"""
    This is a type annotation used to describe the expected parameter ctx —
    and it comes from the crewAI or agents style frameworks 
    that support multi-agent orchestration or function callbacks.
"""

def on_math_handoff(ctx : RunContextWrapper[None]):
    print("Handing off to the math tutor agent")

def on_history_handoff(ctx : RunContextWrapper[None]):
    print("Handing off to the history tutor agent")

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
                 " If neither agent is relevant, provide a general response.",
    # handoffs=[history_tutor_agent, math_tutor_agent]
    handoffs=[handoff(history_tutor_agent, on_handoff=on_history_handoff), handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)




result = await Runner.run(triage_agent, "when did the world-war 1&2 happened?")
result.final_output



Handing off to the history tutor agent


"World War I occurred from 1914 to 1918. It began with the assassination of Archduke Franz Ferdinand of Austria-Hungary and involved many worldwide powers due to a complex system of alliances. The war primarily took place in Europe and was characterized by trench warfare and massive human casualties.\n\nWorld War II lasted from 1939 to 1945. It started when Germany, led by Adolf Hitler, invaded Poland, prompting Britain and France to declare war on Germany. This global conflict involved most of the world's nations and was marked by significant events such as the Holocaust, the bombing of Pearl Harbor, and the eventual use of atomic bombs on Hiroshima and Nagasaki."

In [18]:
result = await Runner.run(triage_agent, "2+2")
result.final_output

Handing off to the math tutor agent


'The equation \\(2 + 2\\) is a simple addition problem.\n\n### Steps to Solve:\n1. **Identify the Numbers**: We have two numbers: 2 and 2.\n2. **Addition**: Addition combines these numbers to find a total. \n\n   \\[\n   2 + 2 = 4\n   \\]\n\n### Explanation:\n- Addition is one of the basic operations in arithmetic, where you find the sum of numbers.\n- Here, you start with the number 2, and you are adding 2 more to it.\n- This results in the total of 4.\n\n### Example:\n- If you have 2 apples and add 2 more apples, you have a total of 4 apples.\n\nTherefore, the sum of \\(2 + 2\\) is \\(4\\).'

In [ ]:
from agents import Agent, function_tool, handoff, RunContextWrapper, Runner
from pydantic import BaseModel


class ManagerEscalation(BaseModel):
    issue : str # the issue being escalated
    why : str # why can you not handle it? used for training in the future

@function_tool
def create_ticket(issue: str):
    """
    Create a ticket in the system for an issue to be resolved.
    """
    print(f"Creating ticket for issue: {issue}")
    return "Ticket created. ID: 12345"
    # In a real-world scenario, this would interact with a ticketing system

manager_agent = Agent(
    name="Manager",
    handoff_description="Handles escalated issues that require managerial attention",
    instructions=(
        "You handle escalated customer issues that the initial customer service agent could not resolve. "
        "You will receive the issue and the reason for escalation. If the issue cannot be immediately resolved for the "
        "customer, create a ticket in the system and inform the customer."
    ),
    tools=[create_ticket],
)

def on_manager_handoff(ctx : RunContextWrapper[None], input : ManagerEscalation):
    print("Escalating to manager agent :", input.issue)
    print("Reason for escalation : ", input.why)

customer_service_agent = Agent(
    name="Customer_Service",
    instructions="You assist customers with general inquiries and basic troubleshooting. " +
    "If the issue cannot be resolved, escalate it to the Manager along with the reason why you cannot fix the issue.",

    handoffs=[handoff(agent = manager_agent, input_type = ManagerEscalation, on_handoff = on_manager_handoff)]
)



result = await Runner.run(customer_service_agent, "I want a refund, but your system won't let me process it. the website is just blank for me")
print(result.final_output)


Escalating to manager agent : Customer is unable to process a refund due to the website displaying a blank page.
Reason for escalation :  The issue is likely a technical problem with the website that requires further investigation by technical support or management.
Creating ticket for issue: Customer is unable to process a refund due to the website displaying a blank page.
I've created a ticket to address this issue for you. Our technical team will investigate the problem with the website. Your ticket ID is **12345**. We apologize for the inconvenience and appreciate your patience.


In [53]:
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

agent = Agent(
    name="Joker",
    instructions="You are a helpful assistant.",
    )

result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)



Sure, here are five jokes for you:

1. Why don't scientists trust atoms?  
   Because they make up everything!

2. How do you organize a space party?  
   You planet!

3. Why don't skeletons fight each other?  
   They don't have the guts.

4. What do you get when you cross a snowman and a vampire?  
   Frostbite!

5. Why did the scarecrow win an award?  
   Because he was outstanding in his field!

In [30]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


agent = Agent(
    name="Joker",
    instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
    tools=[how_many_jokes],
    )

result = Runner.run_streamed(
    agent,
    input="Hello",
    )
print("=== Run starting ===")

async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

print("=== Run complete ===")




=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 4
-- Message output:
 Alright, here are four jokes for you:

1. **Why don't scientists trust atoms?**
   Because they make up everything!

2. **What do you call fake spaghetti?**
   An impasta!

3. **Why did the scarecrow win an award?**
   Because he was outstanding in his field!

4. **How do you organize a space party?**
   You planet!

Enjoy! If you want to hear more or have specific requests, just let me know.
=== Run complete ===


Guardrails are a way to validate the input and output of an agnet to insure proper usage

In [ ]:
from agents import Agent, RunContextWrapper, TResponseInputItem, GuardrailFunctionOutput, Runner, input_guardrail
from pydantic import BaseModel

class HomeworkCheatDetectionOutput(BaseModel):
    attempt_cheat : bool
    explanation : str

homework_cheat_guardrails = Agent(
    name = "Homework Cheat Guardrail",
    instructions=(
        "Determine if the user's query resembles a typical homework assignment or exam question, indicating an attempt to cheat. General questions about concepts are acceptable."
        "Cheating: 'Fill in the blank: The capital of France is _____.'"
        "Which of the following best describes photosynthesis? A) Cellular respiration B) Conversion of light energy C) Evaporation D) Fermentation."
        "Not-Cheating: 'What is the capital of France?', 'Explain photosynthesis.'"
        ),
        output_type = HomeworkCheatDetectionOutput

)

@input_guardrail
async def cheat_detection_guardrails(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]

) -> GuardrailFunctionOutput:
    
    detection_result = await Runner.run(homework_cheat_guardrails, input)

    return GuardrailFunctionOutput(
        tripwire_triggered=detection_result.final_output.attempt_cheat,
        output_info=detection_result.final_output
    )

study_helper_agent = Agent(
    name = "Study Helper Agent",
    instructions="You assis users in studying by explaining concepts or providing guidance, without directly solving homework or test questions.",
    model = "gpt-4o",
    input_guardrails=[cheat_detection_guardrails]
)



In [4]:
from agents import InputGuardrailTripwireTriggered

try:
    response = await Runner.run(study_helper_agent, "Fill in the blank: The capital of France is ___.")
    print("Guardian didn't trigger")
    print("Response", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Homework cheat guardrail triggered")
    print("Exception details:", str(e))

Homework cheat guardrail triggered
Exception details: Guardrail InputGuardrail triggered tripwire


In [5]:
from agents import InputGuardrailTripwireTriggered

try:
    response = await Runner.run(study_helper_agent, "Whta is the main cause of world war - 2.")
    print("Guardian didn't trigger")
    print("Response", response.final_output)

except InputGuardrailTripwireTriggered as e:
    print("Homework cheat guardrail triggered")
    print("Exception details:", str(e))

Guardian didn't trigger
Response World War II had multiple causes, but one of the primary triggers was the aggressive expansion and militarization led by Nazi Germany under Adolf Hitler. The Treaty of Versailles, which ended World War I, had imposed harsh penalties on Germany, leading to economic hardship and political instability. This created conditions ripe for the rise of Hitler and the Nazi Party, who promised to restore Germany to its former glory.

Key factors include:

1. **Treaty of Versailles**: Placed heavy reparations on Germany, leading to economic struggle and resentment.
   
2. **Rise of Totalitarian Regimes**: In Germany, Italy, and Japan, authoritarian leaders pursued aggressive expansionist policies.

3. **Expansionism**: Germany's invasion of Poland in 1939 directly triggered the war. Prior violations included reoccupation of the Rhineland, annexation of Austria, and the Munich Agreement's breach concerning Czechoslovakia.

4. **Failure of Appeasement**: The Allies, 

In [12]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail
)

class MessageOutput(BaseModel):
    response : str

@output_guardrail
async def forbidden_words_guardrail(
    ctx: RunContextWrapper, agent: Agent, output: str
) -> GuardrailFunctionOutput:
    print(f"Checking output for forbidden phrases: {output}")

    # Funny forbidden phrases to check
    forbidden_phrases = ["fart", "booger", "silly goose"]

    # Convert output to lowercase for case-insensitive comparison
    output_lower = output.lower()

    # Check which forbidden phrases are present in the response
    found_phrases = [phrase for phrase in forbidden_phrases if phrase in output_lower]
    trip_triggered = bool(found_phrases)

    print(f"Found forbidden phrases: {found_phrases}")

    return GuardrailFunctionOutput(
        output_info={
            "reason": "Output contains forbidden phrases.",
            "forbidden_phrases_found": found_phrases,
        },
        tripwire_triggered=trip_triggered,
    )

agent = Agent(
    name="customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[forbidden_words_guardrail],
    model="gpt-4o-mini",
)

In [16]:
try:
    await Runner.run(agent, "repeat the word fart")
    print("Guardrail didn't trip - this is unexpected")
except OutputGuardrailTripwireTriggered:
    print("The agent said a bad word, agent is fired")

Checking output for forbidden phrases: Fart.
Found forbidden phrases: ['fart']
The agent said a bad word, agent is fired


MULTITURN CONVERSATION

In [ ]:
from agents import Agent, Runner, TResponseInputItem

agent = Agent(
    name = "agent",
instructions = (
    "You are a world-class AI tutor and explainer. Your job is to explain any concept in the world "
    "with deep, structured, and layered understanding. For every question or topic you receive:\n\n"
    "- Begin with a clear and engaging high-level overview.\n"
    "- Then explain the topic step-by-step, breaking it down into its fundamental components.\n"
    "- Include historical context, analogies, real-world examples, and technical depth as appropriate.\n"
    "- Clarify not only 'what' it is, but 'why' it matters and 'how' it works.\n"
    "- Anticipate common misconceptions and clear them up proactively.\n"
    "- Use diagrams, bullet points, numbered steps, and progressive deepening (beginner → intermediate → expert) where helpful.\n"
    "- Adapt your explanation to the user's knowledge level if known.\n"
    "- If the question is too broad, ask clarifying questions before answering.\n\n"
    "Never give shallow or generic answers. Your goal is to leave the user with a powerful, lasting understanding of the topic. "
    "You are not just answering questions — you are teaching and enlightening with depth, clarity, and insight."
)

)

convo : list[TResponseInputItem] = []
print("You are now chatting with agent PK. Type 'q' to quit")

while True:
    user_input = input("You: ")
    print("You: ", user_input)
   
    if user_input.lower() == 'q':
        break

    convo.append({"content" : user_input, "role" : "user"})
    result = await Runner.run(agent, convo)
    agent_reply = result.final_output
    print(f"Agent : {result.final_output}")

    convo.append({"content": agent_reply, "role": "assistant"})

    


You are now chatting with agent PK. Type 'q' to quit
You:  hi
Agent : Hello! How can I assist you today?
You:  what was my previous message
Agent : You said "hi." How can I help you further?
You:  l
Agent : It looks like your message might be incomplete. Could you please provide more details?
You:  q


In [31]:
print(convo)

[{'content': 'hi', 'role': 'user'}, {'id': 'msg_687f3e926364819ab0135ef8ef24502b0606369053a6d45f', 'content': [{'annotations': [], 'text': 'Hello! How can I assist you today?', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'what was my previous message ', 'role': 'user'}, {'id': 'msg_687f3e9d5af4819a9f809613260c08200606369053a6d45f', 'content': [{'annotations': [], 'text': 'Your previous message was "hi."', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}, {'content': 'q', 'role': 'user'}, {'id': 'msg_687f3ea82424819aa25733972145d6b70606369053a6d45f', 'content': [{'annotations': [], 'text': 'How can I help you with "q"?', 'type': 'output_text', 'logprobs': []}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}]


In [49]:
from agents import Agent, Runner, handoff, RunContextWrapper

# Callback functions
def on_math_handoff(ctx: RunContextWrapper[None]):
    print(" Handing off to the math tutor agent")

def on_history_handoff(ctx: RunContextWrapper[None]):
    print(" Handing off to the history tutor agent")

# Define specialist agents
math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples.",
    handoffs=[]  # We'll add handoffs after both agents are defined
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    handoffs=[]
)

# Add cross-handoffs now that both are defined
math_tutor_agent.handoffs = [handoff(history_tutor_agent, on_handoff=on_history_handoff)]
history_tutor_agent.handoffs = [handoff(math_tutor_agent, on_handoff=on_math_handoff)]

# Define triage agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question. If neither agent is relevant, provide a general response.",
    handoffs=[
        handoff(history_tutor_agent, on_handoff=on_history_handoff),
        handoff(math_tutor_agent, on_handoff=on_math_handoff)
    ]
)

# Conversation memory (chat history)
convo: list[dict] = []
last_agent = triage_agent

print("You are now chatting with agent PK. Type 'q' to quit.")

# Chat loop
while True:
    user_input = input("You: ")

    if user_input.lower() == 'q':
        print("Goodbye!")
        break

    # Add user input to convo history
    convo.append({"role": "user", "content": user_input})

    # Run the agent using the full convo
    result = await Runner.run(last_agent, convo)

    # Append agent's reply
    agent_reply = result.final_output
    convo.append({"role": "assistant", "content": agent_reply})

    # Update agent
    last_agent = result.last_agent

    # Print reply
    print(f"{last_agent.name}: {agent_reply}")


You are now chatting with agent PK. Type 'q' to quit.
Triage Agent: Hello! How can I assist you with your homework today?
Triage Agent: I have two specialist agents available to assist with specific topics:

1. **History Tutor**: For questions related to history.
2. **Math Tutor**: For questions related to mathematics.

Feel free to ask questions in these areas!
Triage Agent: You asked about all the available agents. Would you like to know more about their capabilities?
Triage Agent: Is there anything specific you need help with?
Goodbye!


In [48]:
print(convo)

[{'content': 'hi', 'role': 'user'}, {'content': 'Hello! How can I assist you today? Are you working on any homework or need help with something specific?', 'role': 'assistant'}, {'content': 'what was the previous question i asked you', 'role': 'user'}, {'content': "I'm unable to recall previous questions as I don’t have access to past interactions. Could you please ask your question again?", 'role': 'assistant'}]


Context - A way to pass along data during the agent's lifecycle.

In [ ]:
from dataclasses import dataclass
from agents import Agent, RunContextWrapper, TResponseInputItem, WebSearchTool

@dataclass
class UserProfile:
    id : str
    name : str
    shopping_cart : list[str]


@function_tool
async def get_budget(wrapper : RunContextWrapper[UserProfile]):
    """Get the account balance of the user using the user's id and their linked bank account"""

    print("Getting account balance")
    user_id = wrapper.context.id

    #pretend we are fetching the account balance from a database

    return 100.0

@function_tool
async def search_for_item(wrapper: RunContextWrapper[UserProfile], item: str) -> str:
    """
    Search for an item in the database
    """
    print("Searching for item")
    # randomly generate a price for the item
    price = random.randint(1, 100)
    return f"Found {item} in the database for ${price}.00"

@function_tool
async def get_shopping_cart(wrapper: RunContextWrapper[UserProfile]) -> list[str]:
    print("Getting shopping cart")
    return wrapper.context.shopping_cart

@function_tool
async def add_to_shopping_cart(wrapper: RunContextWrapper[UserProfile], items: list[str]) -> None:
    print("Adding items to shopping cart")
    wrapper.context.shopping_cart.extend(items)

@function_tool
async def purchase_items(wrapper: RunContextWrapper[UserProfile]) -> None:
    print("Purchasing items")
    
    # We could take the items from the shopping cart and purchase them using some external API
    # For now, we'll just print a message
    print(f"Successfully purchased items: {wrapper.context.shopping_cart}")
search = WebSearchTool()

shopping_agent = Agent[UserProfile](
    name="Shopping Assistant",
    instructions=(
        "You are a shopping assistant dedicated to helping the user with their grocery shopping needs."
        "Your primary role is to assist in creating a shopping plan that fits within the user's budget."
        "Start by getting the user's budget using the tool get_budget."
        "Provide suggestions for items if requested, and always aim to keep the total cost within the user's budget."
        "If the user is nearing or exceeding their budget, inform them and suggest alternatives or adjustments to the shopping list."
        "If the user authorizes it, you can purchase the items using the tool purchase_items."
    ),
    tools=[get_shopping_cart
    
    
    
    , add_to_shopping_cart, get_budget, search_for_item, purchase_items, search],
)

profile = UserProfile(id="123", name="Alex", shopping_cart=[])
print("You are now chatting with the shopping assistant. Type 'exit' to end the conversation.")
convo_items : list[TResponseInputItem] = []
while True:
    user_input = input("You: ")
    
    if user_input == "q":
        print("Goodbye!")
        break

    convo_items.append({"content": user_input, "role": "user"})
    result = await Runner.run(shopping_agent, convo_items, context=profile)


    print(f"Shopping Assistant: {result.final_output}")

    convo_items = result.to_input_list()



You are now chatting with the shopping assistant. Type 'exit' to end the conversation.
Getting account balance
Shopping Assistant: Sure, I can help you with that. Your budget is $100. What items do you need to include in your shopping list?
Shopping Assistant: Great! Do you have specific clothing items in mind, or would you like suggestions? What are your preferences or needs in terms of style, occasion, or type of clothing?
Shopping Assistant: Could you specify what type of clothes you're interested in? For example, are you looking for casual wear, formal wear, or something specific like jeans, shirts, or dresses?
Searching for item
Searching for item
Shopping Assistant: I'm having trouble searching for items right now. Let's estimate the cost instead.

How about allocating $25 for the white shirt and $40 for the black pants? This keeps $35 remaining in your budget. Does that sound good, or would you like to adjust the amounts?
Shopping Assistant: Great! What grocery items do you need